In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
con = sqlite3.connect("../data/Checking_Logs.sqlite")

## Get the schema of the table test

In [3]:
pd.io.sql.read_sql("PRAGMA table_info(test);", con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## Get only the first 10 rows of the table test to check what the table looks like

In [4]:
pd.read_sql('SELECT * FROM TEST LIMIT 10', con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [5]:
pd.read_sql('SELECT * FROM DEADLINES', con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


## Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
- do this by joining the table with the table deadlines

- the difference should be displayed in hours

- do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier

- the value should be stored in the dataframe df_min with the corresponding uid

In [8]:
query = """
    SELECT 
        t.uid,
        (d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0 AS delta_hours
    FROM 
        test t
    JOIN 
        deadlines d ON t.labname = d.labs
    WHERE 
        t.labname != 'project1'
    ORDER BY 
        delta_hours ASC
    LIMIT 1;
"""

df_min = pd.read_sql(query, con)
df_min

,uid,delta_hours
0,user_25,2.8675


## Do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [9]:
query = """
    SELECT 
    t.uid,
    (d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0 AS delta_hours
    FROM 
        test t
    JOIN 
        deadlines d ON t.labname = d.labs
    WHERE 
        t.labname != 'project1'
    ORDER BY 
        delta_hours DESC
    LIMIT 1;
"""

df_max = pd.read_sql(query, con)
df_max

,uid,delta_hours
0,user_30,202.385


## Do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [12]:
query = """
    SELECT
        AVG((d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) as average
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
"""
df_avg = pd.read_sql(query, con)
df_avg

,average
0,89.687841


## We want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference
  - using only one query, create a table with the columns: uid, avg_diff, pageviews

  - uid is the uids that exist in the test

  - avg_diff is the average delta between the first commit and the lab deadline per user

  - pageviews is the number of Newsfeed visits per user

  - do not take the lab ’project1’ into account

  - store it to the dataframe views_diff
  
  - use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [21]:
query = """
    SELECT
        t.uid,
        AVG((d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_diff,
        pv.pageviews as pageviews
    FROM test t
    LEFT JOIN deadlines d ON t.labname = d.labs
    LEFT JOIN (
        SELECT
            uid,
            count(*) as pageviews
        FROM pageviews
        GROUP BY uid
    ) pv ON t.uid =  pv.uid
    WHERE t.labname != 'project1'
    GROUP BY t.uid
"""

views_diff = pd.read_sql(query, con)
views_diff

,uid,avg_diff,pageviews
0,user_1,65.119778,28
1,user_10,75.242444,89
2,user_14,159.568796,143
3,user_17,62.207667,47
4,user_18,6.368148,3
5,user_19,99.440417,16
6,user_21,96.111181,10
7,user_25,93.474944,179
8,user_28,86.793833,149
9,user_3,105.738222,317


In [23]:
corr_coef = views_diff[['pageviews', 'avg_diff']].corr().iloc[1, 0]
corr_coef

np.float64(0.27914346425426556)

## Close the connection

In [24]:
con.close()